# SCORING EN TIEMPO REAL - DETECCIÓN DE FRAUDE TELEFÓNICO

### Este notebook procesa CSVs en tiempo real y genera resultados

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime
import warnings
import time
import shutil
warnings.filterwarnings('ignore')

print("✅ Sistema de Scoring en Tiempo Real iniciado")
print(f"🕐 Hora de inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Sistema de Scoring en Tiempo Real iniciado
🕐 Hora de inicio: 2025-06-17 21:25:35


# # 1. CONFIGURACIÓN Y RUTAS

In [2]:
# Rutas principales
MODELS_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Modelos"
INPUT_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Datos_Tiempo_Real"  # Carpeta donde llegan los CSVs
OUTPUT_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Resultados_Tiempo_Real"  # Carpeta para resultados
PROCESSED_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Datos_Procesados"  # Carpeta para CSVs ya procesados
LOG_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Logs"  # Carpeta para logs

# Crear directorios si no existen
for path in [INPUT_PATH, OUTPUT_PATH, PROCESSED_PATH, LOG_PATH]:
    os.makedirs(path, exist_ok=True)

# Configuración de procesamiento
INTERVALO_SEGUNDOS = 180  # 3 minutos = 180 segundos
ARCHIVO_ENTRADA = "datos_dia_actual.csv"  # Nombre del archivo que se actualiza
PREFIJO_SALIDA = "resultados"  # Prefijo para archivos de salida

print(f"📁 Carpeta de entrada: {INPUT_PATH}")
print(f"📁 Carpeta de salida: {OUTPUT_PATH}")
print(f"⏱️ Intervalo de procesamiento: {INTERVALO_SEGUNDOS} segundos")
print(f"📄 Archivo a monitorear: {ARCHIVO_ENTRADA}")

📁 Carpeta de entrada: C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Datos_Tiempo_Real
📁 Carpeta de salida: C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Resultados_Tiempo_Real
⏱️ Intervalo de procesamiento: 180 segundos
📄 Archivo a monitorear: datos_dia_actual.csv


# # 2. CARGAR MODELO Y CONFIGURACIÓN

In [3]:
print("\n🔄 Cargando modelo y configuración...")

try:
    # Cargar modelo
    modelo_path = os.path.join(MODELS_PATH, "modelo_general.pkl")
    with open(modelo_path, 'rb') as f:
        modelo = pickle.load(f)
    
    # Cargar scaler
    scaler_path = os.path.join(MODELS_PATH, "scaler_general.pkl")
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    
    # Cargar configuración
    config_path = os.path.join(MODELS_PATH, "config_modelo_general.pkl")
    with open(config_path, 'rb') as f:
        config = pickle.load(f)
    
    # Extraer componentes importantes
    stats_dict = config['stats_por_pais']
    contexto_historico = config.get('contexto_historico', {})
    umbral_global = config['umbral_global']
    parametros_features = config['parametros_features']
    
    print("✅ Modelo cargado exitosamente")
    print(f"📊 Configuración:")
    print(f"   - Umbral de anomalía: {umbral_global:.4f}")
    print(f"   - Países en contexto: {len(contexto_historico)}")
    print(f"   - Fecha de entrenamiento: {config.get('fecha_entrenamiento', 'No disponible')}")
    
except Exception as e:
    print(f"❌ Error al cargar modelo: {str(e)}")
    raise



🔄 Cargando modelo y configuración...
✅ Modelo cargado exitosamente
📊 Configuración:
   - Umbral de anomalía: 0.9724
   - Países en contexto: 188
   - Fecha de entrenamiento: 2025-06-16T21:08:01.518982


# # 3. FUNCIONES DE PROCESAMIENTO

In [4]:
def crear_features_contextualizadas_mejorada(row, stats_pais_dict):
    """
    Función idéntica a la del modelo original para mantener consistencia
    """
    pais = row['CODIGODEPAIS']
    llamadas = row['N_LLAMADAS']
    minutos = row['N_MINUTOS']
    destinos = row['N_DESTINOS']
    
    # Parámetros de configuración
    PESO_MINUTOS_NORMAL = parametros_features['peso_minutos_normal']
    PESO_MINUTOS_EXTREMOS = parametros_features['peso_minutos_extremos']
    UMBRAL_MINUTOS_EXTREMOS = parametros_features['umbral_minutos_extremos']
    PESO_DESTINOS = parametros_features['peso_destinos']
    PESO_SPRAY_RATIO = parametros_features['peso_spray_ratio']
    
    if pais in stats_pais_dict:
        pais_stats = stats_pais_dict[pais]
        categoria = pais_stats['CATEGORIA']
        
        llamadas_norm = min(llamadas / max(pais_stats['LLAMADAS_P95'], 1), 1.5)
        destinos_norm = min(destinos / max(pais_stats['DESTINOS_P95'], 1), 1.5)
        
        minutos_p90 = pais_stats.get('MINUTOS_P90', pais_stats['MINUTOS_P95'] * 0.9)
        
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            minutos_norm = min(minutos / max(minutos_p90, 1), 3.0)
            peso_minutos = PESO_MINUTOS_EXTREMOS
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(max(minutos_p90, 1)), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL
    else:
        categoria = 'Muy_Bajo'
        llamadas_norm = min(llamadas / 10, 2.0)
        destinos_norm = min(destinos / 5, 2.0)
        
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            minutos_norm = min(minutos / 50, 3.0)
            peso_minutos = PESO_MINUTOS_EXTREMOS * 1.2
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(60), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL
    
    features = {
        'llamadas_norm': llamadas_norm * 0.8,
        'destinos_norm': destinos_norm * PESO_DESTINOS,
        'minutos_norm': minutos_norm * peso_minutos,
        'diversidad_destinos': min(destinos / max(llamadas, 1), 1.0),
        'spray_ratio': min(destinos / max(llamadas, 1) * PESO_SPRAY_RATIO, 1.0) if destinos >= 5 else 0,
        'minutos_extremos': 1.0 if minutos >= UMBRAL_MINUTOS_EXTREMOS else 0.0,
        'minutos_sospechosos': min((minutos - 200) / 300, 1.0) if minutos > 200 else 0.0,
        'patron_spray_fuerte': 1.0 if (destinos >= 10 and llamadas >= 20) else 0.0,
        'patron_spray_medio': 0.5 if (destinos >= 6 and llamadas >= 12) else 0.0,
        'alta_diversidad': min(destinos / 12, 1) if destinos >= 5 else 0,
        'volumen_llamadas_alto': min((llamadas - 30) / 50, 1) if llamadas > 30 else 0,
        'volumen_destinos_alto': min((destinos - 10) / 20, 1) if destinos > 10 else 0,
        'llamadas_por_destino': min(llamadas / max(destinos, 1) / 5, 1),
        'eficiencia_destinos': min(destinos / max(llamadas * 0.5, 1), 1),
        'factor_pais_bajo': 1.5 if categoria in ['Muy_Bajo', 'Bajo'] else 1.0,
        'factor_pais_alto': 0.9 if categoria in ['Alto', 'Medio'] else 1.0
    }
    
    return features

def predecir_anomalia(row, modelo, scaler, umbral, stats_dict, contexto_historico):
    """
    Realiza la predicción de anomalía para un registro
    """
    # Crear features
    features = crear_features_contextualizadas_mejorada(row, stats_dict)
    
    # Normalizar
    features_scaled = scaler.transform_one(features)
    
    # Obtener score
    score = modelo.score_one(features_scaled)
    
    # Lógica de decisión
    es_anomalia_base = score > umbral
    
    if es_anomalia_base:
        pais = row['CODIGODEPAIS']
        llamadas = row['N_LLAMADAS']
        minutos = row['N_MINUTOS']
        destinos = row['N_DESTINOS']
        
        # Verificar tipo de anomalía
        if minutos >= parametros_features['umbral_minutos_extremos']:
            es_anomalia_final = True
            razon = f"Minutos extremos ({minutos:.1f} min)"
            tipo_anomalia = "MINUTOS_EXTREMOS"
        elif destinos >= 6 and llamadas >= 12:
            es_anomalia_final = True
            razon = "Patrón de spray calling confirmado"
            tipo_anomalia = "SPRAY_CALLING"
        elif llamadas > 50 or destinos > 15:
            es_anomalia_final = True
            razon = "Volumen excepcionalmente alto"
            tipo_anomalia = "VOLUMEN_ALTO"
        elif pais not in stats_dict or stats_dict.get(pais, {}).get('CATEGORIA') in ['Muy_Bajo', 'Bajo']:
            if destinos >= 4 and llamadas >= 8:
                es_anomalia_final = True
                razon = "Actividad sospechosa en país de bajo tráfico"
                tipo_anomalia = "PAIS_BAJO_TRAFICO"
            else:
                es_anomalia_final = False
                razon = "Actividad baja en país de bajo tráfico"
                tipo_anomalia = "NO_ANOMALIA"
        else:
            es_anomalia_final = False
            razon = "No cumple criterios de confirmación"
            tipo_anomalia = "NO_ANOMALIA"
    else:
        es_anomalia_final = False
        razon = "Score bajo umbral"
        tipo_anomalia = "NO_ANOMALIA"
    
    # Determinar contexto del país
    if contexto_historico and pais in contexto_historico:
        tipo_contexto = contexto_historico[pais]
    elif pais in stats_dict:
        tipo_contexto = stats_dict[pais]['CATEGORIA']
    else:
        tipo_contexto = "Muy_Bajo"
    
    return {
        'score': score,
        'umbral': umbral,
        'es_anomalia': es_anomalia_final,
        'tipo_anomalia': tipo_anomalia,
        'tipo_contexto': tipo_contexto,
        'razon_decision': razon
    }

print("🔧 Funciones de procesamiento cargadas")

🔧 Funciones de procesamiento cargadas


# # 4. FUNCIÓN PRINCIPAL DE PROCESAMIENTO

In [5]:
def procesar_archivo_csv(archivo_entrada):
    """
    Procesa un archivo CSV y genera resultados
    """
    timestamp = datetime.now()
    timestamp_str = timestamp.strftime("%Y%m%d_%H%M%S")
    
    print(f"\n{'='*60}")
    print(f"🔄 INICIANDO PROCESAMIENTO - {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*60}")
    
    try:
        # Verificar si existe el archivo
        if not os.path.exists(archivo_entrada):
            print(f"⚠️ Archivo no encontrado: {archivo_entrada}")
            return None
        
        # Cargar datos
        print(f"📄 Cargando archivo: {archivo_entrada}")
        df = pd.read_csv(archivo_entrada)
        
        # Validar columnas requeridas
        columnas_requeridas = ['FECHA', 'CODIGODEPAIS', 'LINEA', 'N_LLAMADAS', 'N_MINUTOS', 'N_DESTINOS']
        columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
        
        if columnas_faltantes:
            print(f"❌ Error: Faltan columnas requeridas: {columnas_faltantes}")
            return None
        
        # Convertir fecha
        df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y', errors='coerce')
        
        print(f"📊 Datos cargados:")
        print(f"   - Registros: {len(df)}")
        print(f"   - Países únicos: {df['CODIGODEPAIS'].nunique()}")
        print(f"   - Líneas únicas: {df['LINEA'].nunique()}")
        print(f"   - Rango de fechas: {df['FECHA'].min()} a {df['FECHA'].max()}")
        
        # Procesar cada registro
        print(f"\n🎯 Aplicando modelo de detección...")
        resultados = []
        anomalias_detectadas = 0
        
        for idx, row in df.iterrows():
            if idx % 1000 == 0 and idx > 0:
                print(f"   Procesados: {idx}/{len(df)} registros...")
            
            # Realizar predicción
            resultado = predecir_anomalia(row, modelo, scaler, umbral_global, stats_dict, contexto_historico)
            
            # Agregar información completa
            resultado_completo = {
                'FECHA': row['FECHA'],
                'CODIGODEPAIS': row['CODIGODEPAIS'],
                'LINEA': row['LINEA'],
                'N_LLAMADAS': row['N_LLAMADAS'],
                'N_MINUTOS': row['N_MINUTOS'],
                'N_DESTINOS': row['N_DESTINOS'],
                'score_anomalia': round(resultado['score'], 4),
                'umbral': round(resultado['umbral'], 4),
                'es_anomalia': resultado['es_anomalia'],
                'tipo_anomalia': resultado['tipo_anomalia'],
                'tipo_contexto': resultado['tipo_contexto'],
                'razon_decision': resultado['razon_decision'],
                'timestamp_procesamiento': timestamp
            }
            
            resultados.append(resultado_completo)
            
            if resultado['es_anomalia']:
                anomalias_detectadas += 1
        
        # Crear DataFrame de resultados
        df_resultados = pd.DataFrame(resultados)
        
        # Generar nombre de archivo de salida
        archivo_salida = os.path.join(OUTPUT_PATH, f"{PREFIJO_SALIDA}_{timestamp_str}.csv")
        
        # Guardar resultados completos
        df_resultados.to_csv(archivo_salida, index=False)
        print(f"\n💾 Resultados guardados: {archivo_salida}")
        
        # Guardar solo anomalías si existen
        if anomalias_detectadas > 0:
            df_anomalias = df_resultados[df_resultados['es_anomalia'] == True]
            archivo_anomalias = os.path.join(OUTPUT_PATH, f"anomalias_{timestamp_str}.csv")
            df_anomalias.to_csv(archivo_anomalias, index=False)
            print(f"🚨 Anomalías guardadas: {archivo_anomalias}")
        
        # Resumen de resultados
        print(f"\n📊 RESUMEN DE PROCESAMIENTO:")
        print(f"   - Total registros procesados: {len(df_resultados)}")
        print(f"   - Anomalías detectadas: {anomalias_detectadas}")
        print(f"   - Tasa de anomalías: {(anomalias_detectadas/len(df_resultados)*100):.2f}%")
        
        if anomalias_detectadas > 0:
            print(f"\n🎯 DISTRIBUCIÓN DE ANOMALÍAS:")
            distribucion = df_anomalias['tipo_anomalia'].value_counts()
            for tipo, cantidad in distribucion.items():
                print(f"   - {tipo}: {cantidad} ({cantidad/anomalias_detectadas*100:.1f}%)")
        
        # Crear registro de log
        log_entry = {
            'timestamp': timestamp,
            'archivo_entrada': os.path.basename(archivo_entrada),
            'registros_procesados': len(df_resultados),
            'anomalias_detectadas': anomalias_detectadas,
            'tasa_anomalias': anomalias_detectadas/len(df_resultados)*100,
            'archivo_salida': os.path.basename(archivo_salida)
        }
        
        # Guardar log
        log_file = os.path.join(LOG_PATH, f"log_procesamiento_{timestamp.strftime('%Y%m%d')}.csv")
        if os.path.exists(log_file):
            df_log = pd.read_csv(log_file)
            df_log = pd.concat([df_log, pd.DataFrame([log_entry])], ignore_index=True)
        else:
            df_log = pd.DataFrame([log_entry])
        df_log.to_csv(log_file, index=False)
        
        print(f"📝 Log actualizado: {log_file}")
        
        return df_resultados
        
    except Exception as e:
        print(f"❌ Error durante el procesamiento: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# # 5. MODO DE PROCESAMIENTO ÚNICO (MANUAL)


In [ ]:
# Para procesamiento manual de un archivo específico
print("\n🔧 MODO DE PROCESAMIENTO ÚNICO")

# Archivo de entrada
archivo_manual = os.path.join(INPUT_PATH, ARCHIVO_ENTRADA)

# Verificar si existe
if os.path.exists(archivo_manual):
    print(f"✅ Archivo encontrado: {archivo_manual}")
    
    # Procesar
    resultados = procesar_archivo_csv(archivo_manual)
    
    if resultados is not None:
        print(f"\n✅ Procesamiento completado exitosamente")
        
        # Mover archivo a procesados (opcional)
        archivo_procesado = os.path.join(PROCESSED_PATH, f"procesado_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{ARCHIVO_ENTRADA}")
        shutil.copy2(archivo_manual, archivo_procesado)
        print(f"📁 Archivo copiado a procesados: {archivo_procesado}")
else:
    print(f"⚠️ No se encontró el archivo: {archivo_manual}")
    print(f"📁 Verificar en: {INPUT_PATH}")

# # 6. MODO DE MONITOREO CONTINUO (AUTOMÁTICO)

In [ ]:
def monitoreo_continuo(max_iteraciones=None):
    """
    Monitorea continuamente la carpeta de entrada y procesa archivos
    
    Args:
        max_iteraciones: Número máximo de iteraciones (None = infinito)
    """
    print(f"\n🔄 INICIANDO MONITOREO CONTINUO")
    print(f"⏱️ Intervalo: {INTERVALO_SEGUNDOS} segundos")
    print(f"📁 Monitoreando: {INPUT_PATH}")
    print(f"📄 Archivo objetivo: {ARCHIVO_ENTRADA}")
    print(f"{'='*60}")
    print("Presiona Ctrl+C para detener el monitoreo\n")
    
    iteracion = 0
    ultimo_modificado = None
    
    try:
        while True:
            iteracion += 1
            
            if max_iteraciones and iteracion > max_iteraciones:
                print(f"\n✅ Alcanzado el límite de {max_iteraciones} iteraciones")
                break
            
            archivo_entrada = os.path.join(INPUT_PATH, ARCHIVO_ENTRADA)
            
            # Verificar si el archivo existe
            if os.path.exists(archivo_entrada):
                # Obtener tiempo de modificación
                tiempo_modificacion = os.path.getmtime(archivo_entrada)
                
                # Procesar solo si el archivo fue modificado
                if ultimo_modificado is None or tiempo_modificacion > ultimo_modificado:
                    print(f"\n🔔 Iteración {iteracion} - Archivo detectado/modificado")
                    
                    # Procesar archivo
                    resultados = procesar_archivo_csv(archivo_entrada)
                    
                    if resultados is not None:
                        ultimo_modificado = tiempo_modificacion
                        
                        # Crear copia con timestamp (opcional)
                        archivo_backup = os.path.join(
                            PROCESSED_PATH, 
                            f"backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{ARCHIVO_ENTRADA}"
                        )
                        shutil.copy2(archivo_entrada, archivo_backup)
                else:
                    print(f"\r⏳ Iteración {iteracion} - Sin cambios en el archivo... (esperando {INTERVALO_SEGUNDOS}s)", end='')
            else:
                print(f"\r⏳ Iteración {iteracion} - Archivo no encontrado... (esperando {INTERVALO_SEGUNDOS}s)", end='')
            
            # Esperar antes de la siguiente verificación
            time.sleep(INTERVALO_SEGUNDOS)
            
    except KeyboardInterrupt:
        print(f"\n\n⏹️ Monitoreo detenido por el usuario")
        print(f"📊 Total de iteraciones: {iteracion}")

# # 7. ANÁLISIS DE RESULTADOS HISTÓRICOS


In [ ]:
def analizar_resultados_historicos():
    """
    Analiza todos los resultados generados hasta el momento
    """
    print(f"\n📊 ANÁLISIS DE RESULTADOS HISTÓRICOS")
    print(f"{'='*60}")
    
    # Buscar todos los archivos de resultados
    archivos_resultados = [f for f in os.listdir(OUTPUT_PATH) if f.startswith(PREFIJO_SALIDA) and f.endswith('.csv')]
    
    if not archivos_resultados:
        print("⚠️ No se encontraron archivos de resultados")
        return
    
    print(f"📄 Archivos encontrados: {len(archivos_resultados)}")
    
    # Cargar y combinar todos los resultados
    dfs = []
    for archivo in archivos_resultados:
        try:
            df = pd.read_csv(os.path.join(OUTPUT_PATH, archivo))
            dfs.append(df)
        except:
            continue
    
    if not dfs:
        print("⚠️ No se pudieron cargar los archivos")
        return
    
    df_total = pd.concat(dfs, ignore_index=True)
    
    # Análisis general
    print(f"\n📊 ESTADÍSTICAS GENERALES:")
    print(f"   - Total registros analizados: {len(df_total):,}")
    print(f"   - Total anomalías detectadas: {df_total['es_anomalia'].sum():,}")
    print(f"   - Tasa promedio de anomalías: {df_total['es_anomalia'].mean()*100:.2f}%")
    print(f"   - Países únicos: {df_total['CODIGODEPAIS'].nunique()}")
    print(f"   - Líneas únicas: {df_total['LINEA'].nunique()}")
    
    # Distribución de tipos de anomalías
    anomalias = df_total[df_total['es_anomalia'] == True]
    if len(anomalias) > 0:
        print(f"\n🎯 DISTRIBUCIÓN DE ANOMALÍAS:")
        dist_tipos = anomalias['tipo_anomalia'].value_counts()
        for tipo, cantidad in dist_tipos.items():
            print(f"   - {tipo}: {cantidad:,} ({cantidad/len(anomalias)*100:.1f}%)")
        
        # Top países con más anomalías
        print(f"\n🌍 TOP 10 PAÍSES CON MÁS ANOMALÍAS:")
        top_paises = anomalias['CODIGODEPAIS'].value_counts().head(10)
        for pais, cantidad in top_paises.items():
            print(f"   - {pais}: {cantidad:,} anomalías")
        
        # Evolución temporal
        if 'timestamp_procesamiento' in df_total.columns:
            df_total['timestamp_procesamiento'] = pd.to_datetime(df_total['timestamp_procesamiento'])
            df_total['hora'] = df_total['timestamp_procesamiento'].dt.hour
            
            print(f"\n⏰ DISTRIBUCIÓN POR HORA DEL DÍA:")
            anomalias_por_hora = df_total.groupby('hora')['es_anomalia'].agg(['sum', 'count', 'mean'])
            anomalias_por_hora.columns = ['anomalias', 'total', 'tasa']
            anomalias_por_hora['tasa'] = anomalias_por_hora['tasa'] * 100
            
            for hora, row in anomalias_por_hora.iterrows():
                if row['total'] > 0:
                    print(f"   - {hora:02d}:00: {int(row['anomalias']):,} anomalías de {int(row['total']):,} ({row['tasa']:.1f}%)")

# Ejecutar análisis
analizar_resultados_historicos()

# # 8. UTILIDADES Y FUNCIONES AUXILIARES

In [ ]:
def limpiar_archivos_antiguos(dias_retener=7):
    """
    Limpia archivos de resultados más antiguos que los días especificados
    """
    from datetime import timedelta
    
    fecha_limite = datetime.now() - timedelta(days=dias_retener)
    archivos_eliminados = 0
    
    print(f"\n🧹 Limpiando archivos anteriores a {fecha_limite.strftime('%Y-%m-%d')}")
    
    # Limpiar resultados
    for archivo in os.listdir(OUTPUT_PATH):
        ruta_completa = os.path.join(OUTPUT_PATH, archivo)
        if os.path.isfile(ruta_completa):
            fecha_modificacion = datetime.fromtimestamp(os.path.getmtime(ruta_completa))
            if fecha_modificacion < fecha_limite:
                os.remove(ruta_completa)
                archivos_eliminados += 1
    
    # Limpiar procesados
    for archivo in os.listdir(PROCESSED_PATH):
        ruta_completa = os.path.join(PROCESSED_PATH, archivo)
        if os.path.isfile(ruta_completa):
            fecha_modificacion = datetime.fromtimestamp(os.path.getmtime(ruta_completa))
            if fecha_modificacion < fecha_limite:
                os.remove(ruta_completa)
                archivos_eliminados += 1
    
    print(f"✅ Archivos eliminados: {archivos_eliminados}")

def verificar_salud_sistema():
    """
    Verifica el estado del sistema de procesamiento
    """
    print(f"\n🏥 VERIFICACIÓN DE SALUD DEL SISTEMA")
    print(f"{'='*60}")
    
    # Verificar modelo
    print(f"🤖 Modelo:")
    print(f"   - Cargado: {'✅' if 'modelo' in globals() else '❌'}")
    print(f"   - Umbral: {umbral_global:.4f}")
    
    # Verificar directorios
    print(f"\n📁 Directorios:")
    for nombre, ruta in [
        ("Entrada", INPUT_PATH),
        ("Salida", OUTPUT_PATH),
        ("Procesados", PROCESSED_PATH),
        ("Logs", LOG_PATH)
    ]:
        existe = os.path.exists(ruta)
        print(f"   - {nombre}: {'✅' if existe else '❌'} {ruta}")
    
    # Verificar espacio en disco
    import shutil
    stat = shutil.disk_usage(OUTPUT_PATH)
    espacio_gb = stat.free / (1024**3)
    print(f"\n💾 Espacio en disco:")
    print(f"   - Libre: {espacio_gb:.1f} GB")
    print(f"   - Estado: {'✅' if espacio_gb > 1 else '⚠️ Poco espacio'}")
    
    # Verificar archivos recientes
    print(f"\n📄 Archivos recientes:")
    archivos_entrada = len([f for f in os.listdir(INPUT_PATH) if f.endswith('.csv')])
    archivos_salida = len([f for f in os.listdir(OUTPUT_PATH) if f.endswith('.csv')])
    print(f"   - Archivos en entrada: {archivos_entrada}")
    print(f"   - Archivos en salida: {archivos_salida}")
    
    # Verificar último procesamiento
    archivos_log = [f for f in os.listdir(LOG_PATH) if f.startswith('log_procesamiento')]
    if archivos_log:
        ultimo_log = max(archivos_log)
        print(f"\n📝 Último log: {ultimo_log}")

# Ejecutar verificación
verificar_salud_sistema()

# # 9. INSTRUCCIONES DE USO

In [ ]:
print("\n📚 INSTRUCCIONES DE USO")
print("="*60)
print("\n1️⃣ PROCESAMIENTO ÚNICO (Manual):")
print("   - Coloca tu archivo CSV en:", INPUT_PATH)
print("   - El archivo debe llamarse:", ARCHIVO_ENTRADA)
print("   - Ejecuta la celda de 'MODO DE PROCESAMIENTO ÚNICO'")
print("   - Los resultados se guardarán en:", OUTPUT_PATH)

print("\n2️⃣ MONITOREO CONTINUO (Automático):")
print("   - Ejecuta: monitoreo_continuo()")
print("   - El sistema verificará cada", INTERVALO_SEGUNDOS, "segundos")
print("   - Para limitar iteraciones: monitoreo_continuo(max_iteraciones=10)")
print("   - Para detener: presiona Ctrl+C")

print("\n3️⃣ ANÁLISIS DE HISTÓRICOS:")
print("   - Ejecuta: analizar_resultados_historicos()")
print("   - Verás estadísticas de todos los procesamientos")

print("\n4️⃣ MANTENIMIENTO:")
print("   - Para limpiar archivos antiguos: limpiar_archivos_antiguos(dias_retener=7)")
print("   - Para verificar el sistema: verificar_salud_sistema()")

print("\n📄 FORMATO DEL CSV DE ENTRADA:")
print("   - FECHA (formato: DD/MM/YYYY)")
print("   - CODIGODEPAIS")
print("   - LINEA")
print("   - N_LLAMADAS")
print("   - N_MINUTOS")
print("   - N_DESTINOS")

print("\n📊 ARCHIVOS DE SALIDA GENERADOS:")
print("   - resultados_YYYYMMDD_HHMMSS.csv (todos los registros)")
print("   - anomalias_YYYYMMDD_HHMMSS.csv (solo anomalías)")
print("   - log_procesamiento_YYYYMMDD.csv (registro de procesamientos)")

# # 10. EJEMPLO DE EJECUCIÓN RÁPIDA


In [ ]:
# OPCIÓN 1: Procesamiento único
# Descomenta la siguiente línea para procesar una vez
# resultados = procesar_archivo_csv(os.path.join(INPUT_PATH, ARCHIVO_ENTRADA))

# OPCIÓN 2: Monitoreo continuo por 5 iteraciones
# Descomenta la siguiente línea para monitoreo automático
# monitoreo_continuo(max_iteraciones=5)

# OPCIÓN 3: Monitoreo continuo indefinido
# Descomenta la siguiente línea para monitoreo continuo
# monitoreo_continuo()

print("✅ Sistema listo para usar")
print("🔧 Descomenta una de las opciones anteriores para comenzar")